In [22]:
%endpoint http://localhost:5820/MyDB/update
%auth basic spencer 12345
%format any
%display table

PREFIX fhir: <http://hl7.org/fhir/>
PREFIX fhir_us_core: <http://hl7.org/fhir/us/core/StructureDefinition/>
PREFIX fhir_nhs: <https://fhir.hl7.org.uk/STU3/StructureDefinition/>

############################
### MODIFY FHIR ONTOLOGY ###
############################
## These algorithm require properties and their types (ranges) be connected.
## The existing FHIR ontology does not have this feature - all properties have the generic "fhir:Reference"
## type as their range. The reason for this is the FHIR ontology is designed to roundtrip instances of
## FHIR resources, which simply have generic references (e.g. URIs) to other resources. Validation is
## a separate concern, so this schema information is not captured.
##
## To support our algorithms, we must modify the FHIR ontology so the range of each property is its
## fhir:Reference.type instead of a generic fhir:Reference.
##
## Let's use Patient.generalPractitioner as an example.
############################
###
# this is a "simple" version that just uses one type (Practitioner)
###
# modify the property so its range is the expected type (instead of a generic fhir:Reference)
DELETE WHERE { fhir:Patient.generalPractitioner rdfs:range ?o };
INSERT DATA {
  fhir:Patient.generalPractitioner rdfs:range fhir:Practitioner .
};
###
# TODO the "real" property has multiple types (uses a union_)
###
# first create a class that is the expected range of the property
#DELETE WHERE { fhir:Patient.generalPractitioner.range ?p ?o };
#INSERT DATA {
#    fhir:Patient.generalPractitioner.range a           owl:Class ;
#                                           owl:unionOf (fhir:Organization fhir:Practitioner fhir:PractitionerRole) .
#};
## then modify the property so its range is that class (instead of a generic fhir:Reference)
#DELETE WHERE { fhir:Patient.generalPractitioner rdfs:range ?o };
#INSERT DATA {
#  fhir:Patient.generalPractitioner rdfs:range fhir:Patient.generalPractitioner.range .
#};
## finally, modify all uses of the property so they restrict the property using the expected types
#DELETE { ?bnode owl:allValuesFrom ?o }
#WHERE {
#    ?bnode a                 owl:Restriction ;
#           owl:onProperty    fhir:Patient.generalPractitioner ;
#           owl:allValuesFrom ?o .
#} ;
#INSERT { ?bnode owl:allValuesFrom fhir:Patient.generalPractitioner.range }
#WHERE {
#    ?bnode a                 owl:Restriction ;
#           owl:onProperty    fhir:Patient.generalPractitioner ;
#} ;
#DELETE WHERE { fhir:Patient.generalPractitioner rdfs:range ?o };
#INSERT DATA {
#  fhir:Patient.generalPractitioner rdfs:range fhir:Patient.generalPractitioner.range .
#};


#####################################
### ADD PROFILES TO FHIR ONTOLOGY ###
#####################################

# create USCorePatient (at least enough of it for these examples)
DELETE WHERE { fhir:USCorePatient ?p ?o };
DELETE WHERE { fhir_us_core:us-core-patient ?p ?o };
INSERT DATA {
    fhir_us_core:us-core-patient a owl:Class ;
        rdfs:subClassOf fhir:Patient ;
        rdfs:subClassOf [ a                  owl:Restriction ;
                          owl:onProperty     fhir:Patient.name ;
                          owl:maxCardinality 1
                        ] ;
        rdfs:subClassOf [ a                  owl:Restriction ;
                          owl:onProperty     fhir_us_core:us-core-patient.us_core_birthsex ;
                          owl:maxCardinality 1
                        ] .
};

# create NHSPatient (at least enough of it for these examples)
DELETE WHERE { fhir_nhs:CareConnect-Patient-1 ?p ?o };
DELETE WHERE { fhir_nhs:CareConnect-Organization-1 ?p ?o };
DELETE WHERE { fhir_nhs:CareConnect-Practitioner-1 ?p ?o };
DELETE WHERE { fhir_nhs:CareConnect-Patient-1.generalPractitioner-SIMPLE ?p ?o };
DELETE WHERE { fhir_nhs:CareConnect-Patient-1 ?p ?o };
INSERT DATA { fhir_nhs:CareConnect-Organization-1 a owl:Class; rdfs:subClassOf fhir:Organization . } ;
INSERT DATA { fhir_nhs:CareConnect-Practitioner-1 a owl:Class; rdfs:subClassOf fhir:Practitioner . } ;
###
# this is a "simple" attribute that just uses one type (Practitioner)
###
INSERT DATA {
    fhir_nhs:CareConnect-Patient-1.generalPractitioner-SIMPLE
        a owl:ObjectProperty ;
        rdfs:subPropertyOf fhir:Patient.generalPractitioner ;
        rdfs:domain fhir_nhs:CareConnect-Patient-1 ;
        rdfs:range fhir_nhs:CareConnect-Practitioner-1
} ;
INSERT DATA {
    fhir_nhs:CareConnect-Patient-1 a owl:Class ;
        rdfs:subClassOf fhir:Patient ;
        rdfs:subClassOf [ a                  owl:Restriction ;
                          owl:onProperty     fhir:Patient.name ;
                          owl:cardinality 1
                        ] ;
        rdfs:subClassOf [ a                  owl:Restriction ;
                          owl:onProperty     fhir_nhs:CareConnect-Patient-1.generalPractitioner-SIMPLE ;
                          owl:allValuesFrom  fhir_nhs:CareConnect-Practitioner-1 ] .
} ;
###
# TODO - the "real" attribute uses a union
###
#INSERT DATA {
#    fhir_nhs:CareConnect-Patient-1.generalPractitioner.range a          owl:Class ;
#                                                             owl:unionOf (fhir_nhs:CareConnect-Organization-1 fhir_nhs:CareConnect-Practitioner-1) .
#} ;
#INSERT DATA {
#    fhir_nhs:CareConnect-Patient-1.generalPractitioner
#        a owl:ObjectProperty ;
#        rdfs:subPropertyOf fhir:Patient.generalPractitioner ;
#        rdfs:domain fhir_nhs:CareConnect-Patient-1 ;
#        rdfs:range fhir_nhs:CareConnect-Patient-1.generalPractitioner.range
#} ;
#INSERT DATA {
#    fhir_nhs:CareConnect-Patient-1 a owl:Class ;
#        rdfs:subClassOf fhir:Patient ;
#        rdfs:subClassOf [ a                  owl:Restriction ;
#                          owl:onProperty     fhir:Patient.name ;
#                          owl:cardinality 1
#                        ] ;
#        rdfs:subClassOf [ a                  owl:Restriction ;
#                          owl:onProperty     fhir_nhs:CareConnect-Patient-1.generalPractitioner ;
#                          owl:allValuesFrom  fhir_nhs:CareConnect-Patient-1.generalPractitioner.range ] .
#} ;

# create dummy Patient profile illustrating a specific example
DELETE WHERE { fhir:PatientProfileWithNameCardinalityOne ?p ?o };
INSERT DATA {
    fhir:PatientProfileWithNameCardinalityOne a owl:Class ;
        rdfs:subClassOf fhir:Patient ;
        rdfs:subClassOf [ a                  owl:Restriction ;
                          owl:onProperty     fhir:Patient.name ;
                          owl:cardinality 1
                        ] .
} ;

Endpoint set to: http://localhost:5820/MyDB/update
HTTP authentication: method=basic, user=spencer, passwd set
Request format: ANY
Display: table

In [23]:
######################
# ATTRIBUTE PRESENCE #
######################

%endpoint http://localhost:5820/MyDB/query
%auth basic spencer 12345
%format any
%display table

PREFIX fhir: <http://hl7.org/fhir/>

# Get all instances of profile pairs (cls1 cls2) where
# one profile has a property the other doesn't.
SELECT ?cls1 ?cls2 ?p1
WHERE {
    ?cls1 rdfs:subClassOf fhir:Patient .
    ?cls2 rdfs:subClassOf fhir:Patient .
    ?cls1 rdfs:subClassOf [ a      owl:Restriction ;
                                   owl:onProperty ?p1 ;
                          ] .
    FILTER NOT EXISTS {
        ?cls2 rdfs:subClassOf [ a      owl:Restriction ;
                                   owl:onProperty ?p1 ;
                              ] .
    }
}

Endpoint set to: http://localhost:5820/MyDB/query
HTTP authentication: method=basic, user=spencer, passwd set
Request format: ANY
Display: table

cls1,cls2,p1
http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,http://hl7.org/fhir/PatientProfileWithNameCardinalityOne,http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient.us_core_birthsex
http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1,http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient.us_core_birthsex
https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1,http://hl7.org/fhir/PatientProfileWithNameCardinalityOne,https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1.generalPractitioner-SIMPLE
https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1,http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1.generalPractitioner-SIMPLE


In [24]:
#########################
# ATTRIBUTE CARDINALITY #
#########################

%endpoint http://localhost:5820/MyDB/query
%auth basic spencer 12345
%format any 
%display table

PREFIX fhir: <http://hl7.org/fhir/>

# Get all instances of profile pairs (cls1 cls2) where the profiles
# restrict cardinality in a different way.
SELECT DISTINCT ?cls1 ?cls2 ?property ?card_restriction_1 ?card_1 ?card_restriction_2 ?card_2
WHERE {
    ?cls1 rdfs:subClassOf fhir:Patient .
    ?cls2 rdfs:subClassOf fhir:Patient .
    fhir:Patient rdfs:subClassOf [ a              owl:Restriction ;
                                   owl:onProperty ?property ;
                                 ] .
    ?cls1 rdfs:subClassOf  [ a                  owl:Restriction ;
                             owl:onProperty     ?property ;
                             ?card_restriction_1 ?card_1
                           ] .
    ?cls2 rdfs:subClassOf  [ a                  owl:Restriction ;
                             owl:onProperty     ?property ;
                             ?card_restriction_2 ?card_2
                           ] .
    
    FILTER (?cls1 != ?cls2) .
    FILTER (?card_restriction_1 IN (owl:cardinality, owl:maxCardinality, owl:minCardinality ) ) .
    FILTER (?card_restriction_2 IN (owl:cardinality, owl:maxCardinality, owl:minCardinality ) ) .
    FILTER (?card_restriction_1 != ?card_restriction_2 || ?card_1 != ?card_2) .
}

Endpoint set to: http://localhost:5820/MyDB/query
HTTP authentication: method=basic, user=spencer, passwd set
Request format: ANY
Display: table

cls1,cls2,property,card_restriction_1,card_1,card_restriction_2,card_2
https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1,http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,http://hl7.org/fhir/Patient.name,http://www.w3.org/2002/07/owl#cardinality,1,http://www.w3.org/2002/07/owl#maxCardinality,1
http://hl7.org/fhir/PatientProfileWithNameCardinalityOne,http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,http://hl7.org/fhir/Patient.name,http://www.w3.org/2002/07/owl#cardinality,1,http://www.w3.org/2002/07/owl#maxCardinality,1
http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1,http://hl7.org/fhir/Patient.name,http://www.w3.org/2002/07/owl#maxCardinality,1,http://www.w3.org/2002/07/owl#cardinality,1
http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,http://hl7.org/fhir/PatientProfileWithNameCardinalityOne,http://hl7.org/fhir/Patient.name,http://www.w3.org/2002/07/owl#maxCardinality,1,http://www.w3.org/2002/07/owl#cardinality,1


In [30]:
#####################
# ATTRIBUTE MEANING #
#####################

%endpoint http://localhost:5820/MyDB/query
%auth basic spencer 12345
%format any 
%display table

PREFIX fhir: <http://hl7.org/fhir/>

# Get all instances of profile pairs (cls1 cls2) where the two profiles
# have attributes whose types share a common superclass ancestor.
SELECT DISTINCT ?cls1 ?cls2 ?p1 ?p1_type ?p2 ?p2_type ?p_super ?p_super_type
WHERE {
    ?cls1 rdfs:subClassOf fhir:Patient .
    ?cls2 rdfs:subClassOf fhir:Patient .
    fhir:Patient rdfs:subClassOf [ a              owl:Restriction ;
                                   owl:onProperty ?p_super ;
                                 ] .
    ?cls1 rdfs:subClassOf*/rdfs:subClassOf [ a      owl:Restriction ;
                                             owl:onProperty ?p1 ;
                                           ] .
    ?cls2  rdfs:subClassOf*/rdfs:subClassOf [ a      owl:Restriction ;
                                              owl:onProperty ?p2 ;
                                            ] .
    ?p1 rdfs:range ?p1_type .
    ?p2 rdfs:range ?p2_type .
    ?p_super rdfs:range ?p_super_type .
    ?p1_type rdfs:subClassOf* ?p_super_type .
    ?p2_type rdfs:subClassOf+ ?p_super_type .
    
    FILTER (?cls1 != ?cls2) .
}

Endpoint set to: http://localhost:5820/MyDB/query
HTTP authentication: method=basic, user=spencer, passwd set
Request format: ANY
Display: table

cls1,cls2,p1,p1_type,p2,p2_type,p_super,p_super_type
http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1,http://hl7.org/fhir/Patient.generalPractitioner,http://hl7.org/fhir/Practitioner,https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1.generalPractitioner-SIMPLE,https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Practitioner-1,http://hl7.org/fhir/Patient.generalPractitioner,http://hl7.org/fhir/Practitioner
http://hl7.org/fhir/PatientProfileWithNameCardinalityOne,https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1,http://hl7.org/fhir/Patient.generalPractitioner,http://hl7.org/fhir/Practitioner,https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1.generalPractitioner-SIMPLE,https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Practitioner-1,http://hl7.org/fhir/Patient.generalPractitioner,http://hl7.org/fhir/Practitioner


In [8]:
#####################
# TEST #
#####################

%endpoint http://localhost:5820/MyDB/query
%auth basic spencer 12345
%format any 
%display table

PREFIX fhir: <http://hl7.org/fhir/>

SELECT DISTINCT ?cls1 ?cls2 ?p1 ?p1_type ?p2 ?p2_type ?p_super ?p_super_type
WHERE {
    ?p1 a owl:ObjectProperty .
    ?p2 a owl:ObjectProperty .
    ?p1 rdfs:range ?p1_type .
    ?p2 rdfs:range ?p2_type .
    ?p1_type rdfs:subClassOf+ ?p2_type .
    
    FILTER (?p1 != ?p2) .
}

Endpoint set to: http://localhost:5820/MyDB/query
HTTP authentication: method=basic, user=spencer, passwd set
Request format: ANY
Display: table

cls1,cls2,p1,p1_type,p2,p2_type,p_super,p_super_type
,,http://hl7.org/fhir/CatalogEntry.additionalIdentifier,http://hl7.org/fhir/Identifier,http://hl7.org/fhir/Parameters.parameter.part.value,http://hl7.org/fhir/Element,,
,,http://hl7.org/fhir/CatalogEntry.additionalIdentifier,http://hl7.org/fhir/Identifier,http://hl7.org/fhir/ElementDefinition.pattern,http://hl7.org/fhir/Element,,
,,http://hl7.org/fhir/CatalogEntry.additionalIdentifier,http://hl7.org/fhir/Identifier,http://hl7.org/fhir/ElementDefinition.defaultValue,http://hl7.org/fhir/Element,,
,,http://hl7.org/fhir/CatalogEntry.additionalIdentifier,http://hl7.org/fhir/Identifier,http://hl7.org/fhir/ElementDefinition.fixed,http://hl7.org/fhir/Element,,
,,http://hl7.org/fhir/CatalogEntry.additionalIdentifier,http://hl7.org/fhir/Identifier,http://hl7.org/fhir/ElementDefinition.example.value,http://hl7.org/fhir/Element,,
,,http://hl7.org/fhir/CatalogEntry.additionalIdentifier,http://hl7.org/fhir/Identifier,http://hl7.org/fhir/Task.input.value,http://hl7.org/fhir/Element,,
,,http://hl7.org/fhir/CatalogEntry.additionalIdentifier,http://hl7.org/fhir/Identifier,http://hl7.org/fhir/Parameters.parameter.value,http://hl7.org/fhir/Element,,
,,http://hl7.org/fhir/CatalogEntry.additionalIdentifier,http://hl7.org/fhir/Identifier,http://hl7.org/fhir/Extension.value,http://hl7.org/fhir/Element,,
,,http://hl7.org/fhir/CatalogEntry.additionalIdentifier,http://hl7.org/fhir/Identifier,http://hl7.org/fhir/StructureMap.group.rule.rule.source.defaultValue,http://hl7.org/fhir/Element,,
,,http://hl7.org/fhir/CatalogEntry.additionalIdentifier,http://hl7.org/fhir/Identifier,http://hl7.org/fhir/Task.output.value,http://hl7.org/fhir/Element,,
